In [3]:
import sqlite3
import flask
def MR_DB():
    connection = sqlite3.connect('MRDining.db')
    cursor = connection.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Resturaunts (
            restaurant_id INTEGER PRIMARY KEY AUTOINCREMENT,
            Name TEXT NOT NULL,
            Location TEXT NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Menu (
            Item_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Item_Name TEXT NOT NULL,
            Description TEXT NOT NULL,
            Price REAL NOT NULL,
            restaurant_id INTEGER
            FOREIGN KEY (Item_ID) REFERENCES Restaurants(ID) ON DELETE CASCADE
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Orders (
            Order_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Student_Name TEXT NOT NULL,
            Price REAL NOT NULL,
            restaurant_id INTEGER
            FOREIGN KEY (restaurant_id) REFERENCES Restaurants(restaurant_id) ON DELETE CASCADE
        )
    ''')
    Resturaunt_Data = [
        ("Mozzie's Handcrafted Pizza", "Gander Dining Hall"),
        ("Urban Hen", "Gander Dining Hall"),
        ("Sushi by Faith", "Gander Dining Hall"),
        ("Rice It Up", "Gander Dining Hall"),
        ("Epic Eats", "Gander Dining Hall"),
        ("Louie's", "DUC"),
        ("Starbucks", "Library")
    ]
    cursor.executemany("INSERT INTO Resturaunts (Name, Location) VALUES (?, ?)", Resturaunt_Data)


    connection.commit()
    connection.close()



